In [ ]:
## MNIST 숫자이미지 분석, 우편물 분류를 위해 고안됌.
## Kaggle 에도 데이터가 있고 텐서플로우 안에도 샘플로 데이터가 포함되어 있다.

import tensorflow as tf #텐서플로우의 기본 기능인 그래프를 그리기위해 불러온 것.
from tensorflow.examples.tutorials.mnist import input_data#텐서플로우 인풋데이터라는 함수를 이용해, tf내의 모듈 사용하기위해 불러온 것.
import matplotlib.pyplot as plt
import numpy as np
# 가로 픽셀 세로 픽셀 각 픽셀의 색상, 이미지는 기본 3차원. 그러나 우리가 사용하는 것은 흑백으로 2차원데이터. 28x28 데이터. 

# Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)#원래는 원핫인코딩도 만들어야되는데 tf가 제공해줌.

### plt.imshow(mnist.train.images[0].reshape(28, 28), cmap="Greys")# 이미지쇼, 1차원으로 되어있으므로 이미지쇼를 하려면 2차원으로 리쉐잎해야한다. 
### mnist.train.images[0]# images가 이미지에 대한 데이터를 의미함. 레이블아님.

### print(mnist.train.labels[0])

# placeholder
X = tf.placeholder(shape=[None, 784], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 10], dtype=tf.float32)

# Weight bias
W = tf.Variable(tf.random_normal([784, 10]), name="weight")
b = tf.Variable(tf.random_normal([10]), name="bias")

# Hypothesis
logit = tf.matmul(X, W) + b
H = tf.nn.softmax(logit)#멀티노미널이니까 소프트맥스 사용

# cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=0.09).minimize(cost)
#경사하강법을 계속 써야 하는가? // 애덤 등 옵티마이저 다른 것도 있다. 

# session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
# 기존의 학습은 데이터가 적었기때문에 3000번 진행해도 괜찮았으나, 지금은 데이터가 커서 오래걸림. 
# 따라서 epoch 과 batch 에 대한 개념을 알아보자!!
# epoch : 1 epoch 은 training data를 이용해서 1번 학습하는 것. 3000번은 많음으로 20번으로 수정
train_epoch = 20
batch_size = 100
for step in range(train_epoch):
    num_of_iter = int(mnist.train.num_examples/batch_size)
    
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)# 배치 사이즈 만큼 원본에서 뜯어옴. 
        tmp_train, cost_val = sess.run([train, cost], feed_dict={X:batch_x, Y:batch_y})
        
    if step % 3 == 0:
        print("cost값은 : {}".format(cost_val))

# 정확도 측정
predict = tf.argmax(H, axis=1)
correct = tf.equal(predict, tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

print(sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels })) #원래는 이것도 배치 처리 해야댐.
